In [ ]:
"""
Author: Bryan Bo Cao
Email: boca7588@colorado.edu or bo.cao-1@colorado.edu
Github Repo: https://github.com/BryanBo-Cao/neuralnets-deeplearning
Reference:
    http://www.deeplearning.net/tutorial/lstm.html#lstm
    https://github.com/llSourcell/LSTM_Networks/blob/master/LSTM%20Demo.ipynb
    https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/recurrent_network.py
    Recurrent Neural Network.
    A Recurrent Neural Network (LSTM) implementation example using TensorFlow library.
    This example is using the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/)
    Links:
    [Long Short Term Memory](http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf)
    [MNIST Dataset](http://yann.lecun.com/exdb/mnist/).
    Author: Aymeric Damien
    Project: https://github.com/aymericdamien/TensorFlow-Examples/
"""
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import random
import copy
import matplotlib.pyplot as plt
from importlib import import_module

H = 25
N = 50

#Reference: Denis
def generate_parity_sequences(N, count):
    """
    Generate :count: sequences of length :N:.
    If odd # of 1's -> output 1
    else -> output 0
    """
    xor = lambda x: 1 if (x % 2 == 1) else 0
    sequences = np.random.choice([0, 1], size=[count, N], replace=True)
    counts = np.count_nonzero(sequences == 1, axis=1)
    # xor each sequence, expand dimensions by 1 to match sequences shape
    y = np.expand_dims(np.array([xor(x) for x in counts]), axis=1)

    # In case if you wanted to have the answer just appended at the end of the sequence:
    #     # append the answer at the end of each sequence
    #     seq_plus_y = np.concatenate([sequences, y], axis=1)
    #     print(sequences.shape, y.shape, seq_plus_y.shape)
    #     return seq_plus_y
    return np.expand_dims(sequences, axis=2), y

#Reference: Modified from Denis by Bo Cao
def generate_parity_sequences(N, count):
    """
    Generate :count: sequences of length :N:.
    If odd # of 1's -> output 1
    else -> output 0
    """
    xor = lambda x: 1 if (x % 2 == 1) else 0
    sequences = np.random.choice([0, 1], size=[count, N], replace=True)
    counts = np.count_nonzero(sequences == 1, axis=1)
    # xor each sequence, expand dimensions by 1 to match sequences shape
    y = np.expand_dims(np.array([xor(x) for x in counts]), axis=1)

    # In case if you wanted to have the answer just appended at the end of the sequence:
    #     # append the answer at the end of each sequence
    #     seq_plus_y = np.concatenate([sequences, y], axis=1)
    #     print(sequences.shape, y.shape, seq_plus_y.shape)
    #     return seq_plus_y
    
    new_y = []
    for i in range(len(y)):
        new_yy = []
        if y[i] == 0:
            new_yy.append(0)
            new_yy.append(1)
        else:
            new_yy.append(1)
            new_yy.append(0)
        new_y.append(new_yy)

    return np.expand_dims(sequences, axis=2), new_y

In [ ]:
# Training Parameters
learning_rate = 0.25
batch_size = 128
display_step = batch_size * 100

#batch_steps = 10000 / batch_size
epochs = 5000

# Network Parameters
num_input = 1 # 
timesteps = N # timesteps
num_hidden = H # hidden layer num of features
num_classes = 2 # 0 or 1

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}


def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

logits = RNN(X, weights, biases)
#prediction = tf.nn.softmax(logits)
prediction = tf.tanh(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

test_accuracies_10replications = []
minibatch_losses_1st_replication = [] #epoch as unit
test_accuracies_1st_replication = [] #epoch as unit
train_accuracies_1st_replication = [] #epoch as unit

# Start training
with tf.Session() as sess:
    
    # Run 10 replications
    for replication in range(10):
        
        print("Replication: %d: " % replication)
        
        # Initialize random weights
        train_data = generate_parity_sequences(N, 10000)
        train_data_x = train_data[0]
        train_data_y = train_data[1]
        test_data = generate_parity_sequences(N, 10000)
        test_data_x = test_data[0]
        test_data_y = test_data[1]
        
        # Run the initializer
        sess.run(init)
    
        for epoch in range(epochs):
            batch_index = 0
            while batch_index < 10000:

                train_data_batch_x = []
                train_data_batch_y = []
                if batch_index + batch_size < 10000: 
                    train_data_batch_x = train_data_x[batch_index : batch_index + batch_size]
                    train_data_batch_y = train_data_y[batch_index : batch_index + batch_size]
                else:
                    train_data_batch_x = train_data_x[batch_index : ]
                    train_data_batch_y = train_data_y[batch_index : ]

                #batch_x, batch_y = mnist.train.next_batch(batch_size)
                # Reshape data to get 28 seq of 28 elements
                #batch_x = batch_x.reshape((batch_size, timesteps, num_input))
                #train_data_x = train_data_x.reshape((10000, timesteps, num_input))
                #print("train_data_batch_x.shape:  " , train_data_batch_x.shape)
                if batch_index + batch_size < 10000: 
                    train_data_batch_x = train_data_batch_x.reshape((batch_size, timesteps, num_input))
                else:
                    train_data_batch_x = train_data_batch_x.reshape((10000 % batch_size, timesteps, num_input))
                # Run optimization op (backprop)
                #sess.run(train_op, feed_dict={X: train_data_x, 
                 #                             Y: train_data_y})
                sess.run(train_op, feed_dict={X: train_data_batch_x, 
                                              Y: train_data_batch_y})

                batch_index += batch_size

            if replication == 0:
                loss, train_accuracy = sess.run([loss_op, accuracy], feed_dict={X: train_data_x, Y: train_data_y})
                test_accuracy = sess.run(accuracy, feed_dict={X: test_data_x, Y: test_data_y})
                minibatch_losses_1st_replication.append(loss)
                train_accuracies_1st_replication.append(train_accuracy)
                test_accuracies_1st_replication.append(test_accuracy)
            
            if epoch % 10 == 0:
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: train_data_x,
                                                                         Y: train_data_y})
                print("Epoch: " + str(epoch) + \
                          ", Minibatch Loss= " + \
                          "{:.4f}".format(loss) + ", Training Accuracy= " + \
                          "{:.3f}".format(acc))

        print("Optimization Finished!")

        test_data_x = test_data_x.reshape((-1, timesteps, num_input))
        test_data_y = test_data_y
        test_accuracy = sess.run(accuracy, feed_dict={X: test_data_x, Y: test_data_y})
        test_accuracies_10replications.append(test_accuracy)
        print("Testing Accuracy:", test_accuracy)

Replication: 0: 
Epoch: 0, Minibatch Loss= 0.8090, Training Accuracy= 0.499
Epoch: 10, Minibatch Loss= 0.7167, Training Accuracy= 0.499
Epoch: 20, Minibatch Loss= 0.7074, Training Accuracy= 0.499
Epoch: 30, Minibatch Loss= 0.7035, Training Accuracy= 0.499
Epoch: 40, Minibatch Loss= 0.7013, Training Accuracy= 0.499
Epoch: 50, Minibatch Loss= 0.6998, Training Accuracy= 0.499
Epoch: 60, Minibatch Loss= 0.6988, Training Accuracy= 0.499
Epoch: 70, Minibatch Loss= 0.6981, Training Accuracy= 0.499
Epoch: 80, Minibatch Loss= 0.6975, Training Accuracy= 0.499
Epoch: 90, Minibatch Loss= 0.6971, Training Accuracy= 0.499
Epoch: 100, Minibatch Loss= 0.6967, Training Accuracy= 0.499
Epoch: 110, Minibatch Loss= 0.6964, Training Accuracy= 0.499
Epoch: 120, Minibatch Loss= 0.6962, Training Accuracy= 0.499
Epoch: 130, Minibatch Loss= 0.6959, Training Accuracy= 0.499
Epoch: 140, Minibatch Loss= 0.6958, Training Accuracy= 0.499
Epoch: 150, Minibatch Loss= 0.6956, Training Accuracy= 0.499
Epoch: 160, Miniba

Epoch: 1340, Minibatch Loss= 0.6428, Training Accuracy= 0.619
Epoch: 1350, Minibatch Loss= 0.6484, Training Accuracy= 0.611
Epoch: 1360, Minibatch Loss= 0.6966, Training Accuracy= 0.569
Epoch: 1370, Minibatch Loss= 0.6573, Training Accuracy= 0.604
Epoch: 1380, Minibatch Loss= 0.6739, Training Accuracy= 0.581
Epoch: 1390, Minibatch Loss= 0.6342, Training Accuracy= 0.628
Epoch: 1400, Minibatch Loss= 0.6404, Training Accuracy= 0.616
Epoch: 1410, Minibatch Loss= 0.6388, Training Accuracy= 0.622
Epoch: 1420, Minibatch Loss= 0.6550, Training Accuracy= 0.607
Epoch: 1430, Minibatch Loss= 0.6842, Training Accuracy= 0.579
Epoch: 1440, Minibatch Loss= 0.6340, Training Accuracy= 0.630
Epoch: 1450, Minibatch Loss= 0.6430, Training Accuracy= 0.623
Epoch: 1460, Minibatch Loss= 0.6259, Training Accuracy= 0.640
Epoch: 1470, Minibatch Loss= 0.6972, Training Accuracy= 0.549
Epoch: 1480, Minibatch Loss= 0.7349, Training Accuracy= 0.499
Epoch: 1490, Minibatch Loss= 0.7185, Training Accuracy= 0.499
Epoch: 1

Epoch: 2670, Minibatch Loss= 0.7003, Training Accuracy= 0.535
Epoch: 2680, Minibatch Loss= 0.6885, Training Accuracy= 0.550
Epoch: 2690, Minibatch Loss= 0.6849, Training Accuracy= 0.555
Epoch: 2700, Minibatch Loss= 0.7116, Training Accuracy= 0.516
Epoch: 2710, Minibatch Loss= 0.6800, Training Accuracy= 0.559
Epoch: 2720, Minibatch Loss= 0.6804, Training Accuracy= 0.556
Epoch: 2730, Minibatch Loss= 0.6854, Training Accuracy= 0.548
Epoch: 2740, Minibatch Loss= 0.6819, Training Accuracy= 0.559
Epoch: 2750, Minibatch Loss= 0.6783, Training Accuracy= 0.561
Epoch: 2760, Minibatch Loss= 0.6901, Training Accuracy= 0.535
Epoch: 2770, Minibatch Loss= 0.6900, Training Accuracy= 0.537
Epoch: 2780, Minibatch Loss= 0.7065, Training Accuracy= 0.502
Epoch: 2790, Minibatch Loss= 0.6895, Training Accuracy= 0.536
Epoch: 2800, Minibatch Loss= 0.6906, Training Accuracy= 0.540
Epoch: 2810, Minibatch Loss= 0.6819, Training Accuracy= 0.559
Epoch: 2820, Minibatch Loss= 0.6775, Training Accuracy= 0.569
Epoch: 2

Epoch: 4000, Minibatch Loss= 0.7177, Training Accuracy= 0.513
Epoch: 4010, Minibatch Loss= 0.7176, Training Accuracy= 0.514
Epoch: 4020, Minibatch Loss= 0.7178, Training Accuracy= 0.511
Epoch: 4030, Minibatch Loss= 0.7165, Training Accuracy= 0.511
Epoch: 4040, Minibatch Loss= 0.7162, Training Accuracy= 0.515
Epoch: 4050, Minibatch Loss= 0.7156, Training Accuracy= 0.514
Epoch: 4060, Minibatch Loss= 0.7154, Training Accuracy= 0.515
Epoch: 4070, Minibatch Loss= 0.7134, Training Accuracy= 0.519
Epoch: 4080, Minibatch Loss= 0.7150, Training Accuracy= 0.517
Epoch: 4090, Minibatch Loss= 0.7159, Training Accuracy= 0.517
Epoch: 4100, Minibatch Loss= 0.7247, Training Accuracy= 0.508
Epoch: 4110, Minibatch Loss= 0.7103, Training Accuracy= 0.518
Epoch: 4120, Minibatch Loss= 0.7149, Training Accuracy= 0.519
Epoch: 4130, Minibatch Loss= 0.7091, Training Accuracy= 0.521
Epoch: 4140, Minibatch Loss= 0.7086, Training Accuracy= 0.523
Epoch: 4150, Minibatch Loss= 0.7082, Training Accuracy= 0.524
Epoch: 4

Epoch: 320, Minibatch Loss= 0.6933, Training Accuracy= 0.500
Epoch: 330, Minibatch Loss= 0.6933, Training Accuracy= 0.500
Epoch: 340, Minibatch Loss= 0.6933, Training Accuracy= 0.500
Epoch: 350, Minibatch Loss= 0.6933, Training Accuracy= 0.500
Epoch: 360, Minibatch Loss= 0.6933, Training Accuracy= 0.499
Epoch: 370, Minibatch Loss= 0.6932, Training Accuracy= 0.499
Epoch: 380, Minibatch Loss= 0.6932, Training Accuracy= 0.499
Epoch: 390, Minibatch Loss= 0.6932, Training Accuracy= 0.500
Epoch: 400, Minibatch Loss= 0.6932, Training Accuracy= 0.500
Epoch: 410, Minibatch Loss= 0.6932, Training Accuracy= 0.502
Epoch: 420, Minibatch Loss= 0.6932, Training Accuracy= 0.501
Epoch: 430, Minibatch Loss= 0.6932, Training Accuracy= 0.501
Epoch: 440, Minibatch Loss= 0.6932, Training Accuracy= 0.501
Epoch: 450, Minibatch Loss= 0.6932, Training Accuracy= 0.501
Epoch: 460, Minibatch Loss= 0.6932, Training Accuracy= 0.501
Epoch: 470, Minibatch Loss= 0.6932, Training Accuracy= 0.501
Epoch: 480, Minibatch Lo

...

Epoch: 1930, Minibatch Loss= 0.6860, Training Accuracy= 0.539
Epoch: 1940, Minibatch Loss= 0.6947, Training Accuracy= 0.537
Epoch: 1950, Minibatch Loss= 0.6931, Training Accuracy= 0.514
Epoch: 1960, Minibatch Loss= 0.6913, Training Accuracy= 0.524
Epoch: 1970, Minibatch Loss= 0.6905, Training Accuracy= 0.532
Epoch: 1980, Minibatch Loss= 0.6897, Training Accuracy= 0.534
Epoch: 1990, Minibatch Loss= 0.6885, Training Accuracy= 0.538
Epoch: 2000, Minibatch Loss= 0.6873, Training Accuracy= 0.544
Epoch: 2010, Minibatch Loss= 0.6855, Training Accuracy= 0.550
Epoch: 2020, Minibatch Loss= 0.6850, Training Accuracy= 0.547
Epoch: 2030, Minibatch Loss= 0.6846, Training Accuracy= 0.550
Epoch: 2040, Minibatch Loss= 0.6870, Training Accuracy= 0.549
Epoch: 2050, Minibatch Loss= 0.6853, Training Accuracy= 0.550
Epoch: 2060, Minibatch Loss= 0.6820, Training Accuracy= 0.559
Epoch: 2070, Minibatch Loss= 0.6816, Training Accuracy= 0.561
Epoch: 2080, Minibatch Loss= 0.6832, Training Accuracy= 0.553
Epoch: 2

Epoch: 3260, Minibatch Loss= 0.6945, Training Accuracy= 0.511
Epoch: 3270, Minibatch Loss= 0.6943, Training Accuracy= 0.513
Epoch: 3280, Minibatch Loss= 0.6941, Training Accuracy= 0.511
Epoch: 3290, Minibatch Loss= 0.6942, Training Accuracy= 0.512
Epoch: 3300, Minibatch Loss= 0.6941, Training Accuracy= 0.513
Epoch: 3310, Minibatch Loss= 0.6938, Training Accuracy= 0.512
Epoch: 3320, Minibatch Loss= 0.6936, Training Accuracy= 0.515
Epoch: 3330, Minibatch Loss= 0.6933, Training Accuracy= 0.517
Epoch: 3340, Minibatch Loss= 0.6932, Training Accuracy= 0.515
Epoch: 3350, Minibatch Loss= 0.6929, Training Accuracy= 0.517
Epoch: 3360, Minibatch Loss= 0.6928, Training Accuracy= 0.518
Epoch: 3370, Minibatch Loss= 0.6927, Training Accuracy= 0.516
Epoch: 3380, Minibatch Loss= 0.6926, Training Accuracy= 0.519
Epoch: 3390, Minibatch Loss= 0.6925, Training Accuracy= 0.519
Epoch: 3400, Minibatch Loss= 0.6924, Training Accuracy= 0.520
Epoch: 3410, Minibatch Loss= 0.6923, Training Accuracy= 0.520
Epoch: 3

Epoch: 4590, Minibatch Loss= 0.6944, Training Accuracy= 0.501
Epoch: 4600, Minibatch Loss= 0.6940, Training Accuracy= 0.502
Epoch: 4610, Minibatch Loss= 0.6937, Training Accuracy= 0.504
Epoch: 4620, Minibatch Loss= 0.6936, Training Accuracy= 0.507
Epoch: 4630, Minibatch Loss= 0.6935, Training Accuracy= 0.509
Epoch: 4640, Minibatch Loss= 0.6934, Training Accuracy= 0.509
Epoch: 4650, Minibatch Loss= 0.6934, Training Accuracy= 0.510
Epoch: 4660, Minibatch Loss= 0.6934, Training Accuracy= 0.512
Epoch: 4670, Minibatch Loss= 0.6934, Training Accuracy= 0.512
Epoch: 4680, Minibatch Loss= 0.6934, Training Accuracy= 0.513
Epoch: 4690, Minibatch Loss= 0.6934, Training Accuracy= 0.513
Epoch: 4700, Minibatch Loss= 0.6934, Training Accuracy= 0.512
Epoch: 4710, Minibatch Loss= 0.6933, Training Accuracy= 0.512
Epoch: 4720, Minibatch Loss= 0.6933, Training Accuracy= 0.512
Epoch: 4730, Minibatch Loss= 0.6933, Training Accuracy= 0.510
Epoch: 4740, Minibatch Loss= 0.6933, Training Accuracy= 0.509
Epoch: 4

Epoch: 920, Minibatch Loss= 0.6843, Training Accuracy= 0.545
Epoch: 930, Minibatch Loss= 0.6856, Training Accuracy= 0.543
Epoch: 940, Minibatch Loss= 0.6913, Training Accuracy= 0.527
Epoch: 950, Minibatch Loss= 0.6894, Training Accuracy= 0.531
Epoch: 960, Minibatch Loss= 0.6887, Training Accuracy= 0.534
Epoch: 970, Minibatch Loss= 0.6887, Training Accuracy= 0.535
Epoch: 980, Minibatch Loss= 0.6875, Training Accuracy= 0.535
Epoch: 990, Minibatch Loss= 0.6872, Training Accuracy= 0.537
Epoch: 1000, Minibatch Loss= 0.6882, Training Accuracy= 0.536
Epoch: 1010, Minibatch Loss= 0.7033, Training Accuracy= 0.513
Epoch: 1020, Minibatch Loss= 0.7047, Training Accuracy= 0.508
Epoch: 1030, Minibatch Loss= 0.6984, Training Accuracy= 0.508
Epoch: 1040, Minibatch Loss= 0.6957, Training Accuracy= 0.508
Epoch: 1050, Minibatch Loss= 0.6946, Training Accuracy= 0.508
Epoch: 1060, Minibatch Loss= 0.6940, Training Accuracy= 0.509
Epoch: 1070, Minibatch Loss= 0.6935, Training Accuracy= 0.511
Epoch: 1080, Min

Epoch: 2250, Minibatch Loss= 0.6697, Training Accuracy= 0.582
Epoch: 2260, Minibatch Loss= 0.6834, Training Accuracy= 0.562
Epoch: 2270, Minibatch Loss= 0.6666, Training Accuracy= 0.589
Epoch: 2280, Minibatch Loss= 0.6857, Training Accuracy= 0.554
Epoch: 2290, Minibatch Loss= 0.6651, Training Accuracy= 0.594
Epoch: 2300, Minibatch Loss= 0.6664, Training Accuracy= 0.591
Epoch: 2310, Minibatch Loss= 0.6927, Training Accuracy= 0.557
Epoch: 2320, Minibatch Loss= 0.6677, Training Accuracy= 0.590
Epoch: 2330, Minibatch Loss= 0.6891, Training Accuracy= 0.557
Epoch: 2340, Minibatch Loss= 0.6885, Training Accuracy= 0.562
Epoch: 2350, Minibatch Loss= 0.6647, Training Accuracy= 0.589
Epoch: 2360, Minibatch Loss= 0.6637, Training Accuracy= 0.599
Epoch: 2370, Minibatch Loss= 0.6643, Training Accuracy= 0.596
Epoch: 2380, Minibatch Loss= 0.6993, Training Accuracy= 0.516
Epoch: 2390, Minibatch Loss= 0.6928, Training Accuracy= 0.521
Epoch: 2400, Minibatch Loss= 0.6904, Training Accuracy= 0.527
Epoch: 2

In [ ]:
# print results
test_accuracies_10replications_std = np.std(test_accuracies_10replications, axis=0)
test_accuracies_10replications_std_mean = test_accuracies_10replications_std / np.square(10)
print("test_accuracies_10replications: ", test_accuracies_10replications)
print("mean of test_accuracies_10replications: ", np.mean(test_accuracies_10replications))
print("standard deviation of test_accuracies_10replications_std_mean: ", test_accuracies_10replications_std_mean)

In [ ]:
minibatch_losses_1st_replication
plt.plot(minibatch_losses_1st_replication, color='green', linewidth=5)
plt.plot(train_accuracies_1st_replication, color='blue', linewidth=7)
plt.plot(test_accuracies_1st_replication, color='red', linewidth=3)
plt.xlim(0, epochs)
plt.xlabel("epoch")
plt.ylim(0, 1.2)
plt.ylabel("train acc in blue, test acc in red, minibacth loss in green")
plt.title("1st replication")
plt.show()